# TabPFN Embedding Extraction and Downstream Modeling Workflow

This notebook demonstrates how to:
1. Load a dataset (classification or regression)
2. Extract TabPFN embeddings
3. Use embeddings as features for downstream models (classification and regression)
4. Evaluate performance.

# Prediction Targets in This Workbook
This notebook demonstrates TabPFN embedding extraction and downstream modeling for two tasks:

- **Classification:** Predicting whether a tumor is malignant or benign using the Breast Cancer dataset (`load_breast_cancer`). The target variable is binary: 0 (benign) or 1 (malignant).
- **Regression:** Predicting disease progression (a quantitative measure) using the Diabetes dataset (`load_diabetes`). The target variable is a continuous value representing disease progression one year after baseline.

TabPFN embeddings are extracted for both tasks and used as features for downstream models (logistic regression for classification, linear regression for regression). Performance is evaluated using accuracy (classification) and R² score (regression).

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer, load_diabetes
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import train_test_split
from tabpfn_extensions import TabPFNClassifier, TabPFNRegressor
from tabpfn_extensions.embedding import TabPFNEmbedding

/Users/Scott/Documents/Data Science/ADSWP/TabPFN/.venv/lib/python3.9/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## Classification Example: Breast Cancer Dataset

In [2]:
# Load and split classification data
clf_data = load_breast_cancer()
X_clf, y_clf = clf_data.data, clf_data.target
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42
)

In [6]:
# Baseline logistic regression
baseline_clf = LogisticRegression(max_iter=1000, random_state=42)
baseline_clf.fit(X_train_clf, y_train_clf)
baseline_acc = accuracy_score(y_test_clf, baseline_clf.predict(X_test_clf))
print(f'Baseline Logistic Regression Accuracy: {baseline_acc:.4f}')

Baseline Logistic Regression Accuracy: 0.9561


/Users/Scott/Documents/Data Science/ADSWP/TabPFN/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
clf = TabPFNClassifier(n_estimators=1, random_state=42)
embedding_extractor_clf = TabPFNEmbedding(tabpfn_clf=clf, n_fold=0)
train_embeddings_clf = embedding_extractor_clf.get_embeddings(
    X_train_clf, y_train_clf, X_test_clf, data_source="train"
)[0]
test_embeddings_clf = embedding_extractor_clf.get_embeddings(
    X_train_clf, y_train_clf, X_test_clf, data_source="test"
)[0]

In [8]:
# Downstream classifier on TabPFN embeddings
downstream_clf = LogisticRegression(max_iter=1000, random_state=42)
downstream_clf.fit(train_embeddings_clf, y_train_clf)
y_pred_clf = downstream_clf.predict(test_embeddings_clf)
print(f'TabPFN Embedding + Logistic Regression Accuracy: {accuracy_score(y_test_clf, y_pred_clf):.4f}')

TabPFN Embedding + Logistic Regression Accuracy: 0.9825


## Regression Example: Diabetes Dataset

In [9]:
# Load and split regression data
reg_data = load_diabetes()
X_reg, y_reg = reg_data.data, reg_data.target
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

In [10]:
# Baseline linear regression
baseline_reg = LinearRegression()
baseline_reg.fit(X_train_reg, y_train_reg)
baseline_r2 = r2_score(y_test_reg, baseline_reg.predict(X_test_reg))
print(f'Baseline Linear Regression R2: {baseline_r2:.4f}')

Baseline Linear Regression R2: 0.4526


In [11]:
# TabPFN Embedding extraction for regression
reg = TabPFNRegressor(n_estimators=1, random_state=42)
embedding_extractor_reg = TabPFNEmbedding(tabpfn_clf=reg, n_fold=0)
train_embeddings_reg = embedding_extractor_reg.get_embeddings(
    X_train_reg, y_train_reg, X_test_reg, data_source="train"
 )[0]
test_embeddings_reg = embedding_extractor_reg.get_embeddings(
    X_train_reg, y_train_reg, X_test_reg, data_source="test"
 )[0]

In [12]:
# Downstream regressor on TabPFN embeddings
downstream_reg = LinearRegression()
downstream_reg.fit(train_embeddings_reg, y_train_reg)
y_pred_reg = downstream_reg.predict(test_embeddings_reg)
print(f'TabPFN Embedding + Linear Regression R2: {r2_score(y_test_reg, y_pred_reg):.4f}')

TabPFN Embedding + Linear Regression R2: 0.0506


In [15]:
# Baseline classifier: Random Forest on raw features (classification)
from sklearn.ensemble import RandomForestRegressor
rf_clf_baseline = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf_baseline.fit(X_train_clf, y_train_clf)
y_pred_rf_clf_baseline = rf_clf_baseline.predict(X_test_clf)
print(f'Baseline Random Forest Accuracy (raw features): {accuracy_score(y_test_clf, y_pred_rf_clf_baseline):.4f}')

# Baseline regressor: Random Forest on raw features (regression)
rf_reg_baseline = RandomForestRegressor(n_estimators=100, random_state=42)
rf_reg_baseline.fit(X_train_reg, y_train_reg)
y_pred_rf_reg_baseline = rf_reg_baseline.predict(X_test_reg)
print(f'Baseline Random Forest R2 (raw features): {r2_score(y_test_reg, y_pred_rf_reg_baseline):.4f}')

Baseline Random Forest Accuracy (raw features): 0.9649
Baseline Random Forest R2 (raw features): 0.4428
Baseline Random Forest R2 (raw features): 0.4428


In [16]:
# Downstream classifier: Random Forest on TabPFN embeddings (classification)
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(train_embeddings_clf, y_train_clf)
y_pred_rf_clf = rf_clf.predict(test_embeddings_clf)
print(f'TabPFN Embedding + Random Forest Accuracy: {accuracy_score(y_test_clf, y_pred_rf_clf):.4f}')


# Downstream regressor: Random Forest on TabPFN embeddings (regression)
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
rf_reg.fit(train_embeddings_reg, y_train_reg)
y_pred_rf_reg = rf_reg.predict(test_embeddings_reg)
print(f'TabPFN Embedding + Random Forest R2: {r2_score(y_test_reg, y_pred_rf_reg):.4f}')

TabPFN Embedding + Random Forest Accuracy: 0.9474
TabPFN Embedding + Random Forest R2: -0.0066
TabPFN Embedding + Random Forest R2: -0.0066


In [17]:
# Downstream classifier: MLP on TabPFN embeddings (classification) with scaling
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
scaler_clf = StandardScaler()
X_train_emb_scaled_clf = scaler_clf.fit_transform(train_embeddings_clf)
X_test_emb_scaled_clf = scaler_clf.transform(test_embeddings_clf)
mlp_clf = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
mlp_clf.fit(X_train_emb_scaled_clf, y_train_clf)
y_pred_mlp_clf = mlp_clf.predict(X_test_emb_scaled_clf)
print(f'TabPFN Embedding + MLP Accuracy: {accuracy_score(y_test_clf, y_pred_mlp_clf):.4f}')


# Downstream regressor: MLP on TabPFN embeddings (regression) with scaling
from sklearn.neural_network import MLPRegressor
scaler_reg = StandardScaler()
X_train_emb_scaled_reg = scaler_reg.fit_transform(train_embeddings_reg)
X_test_emb_scaled_reg = scaler_reg.transform(test_embeddings_reg)
mlp_reg = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
mlp_reg.fit(X_train_emb_scaled_reg, y_train_reg)
y_pred_mlp_reg = mlp_reg.predict(X_test_emb_scaled_reg)
print(f'TabPFN Embedding + MLP R2: {r2_score(y_test_reg, y_pred_mlp_reg):.4f}')

TabPFN Embedding + MLP Accuracy: 0.6228
TabPFN Embedding + MLP R2: -3.3782


/Users/Scott/Documents/Data Science/ADSWP/TabPFN/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [18]:
# Baseline classifier: MLP on raw features (classification) with scaling
scaler_clf_raw = StandardScaler()
X_train_clf_scaled = scaler_clf_raw.fit_transform(X_train_clf)
X_test_clf_scaled = scaler_clf_raw.transform(X_test_clf)
mlp_clf_raw = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
mlp_clf_raw.fit(X_train_clf_scaled, y_train_clf)
y_pred_mlp_clf_raw = mlp_clf_raw.predict(X_test_clf_scaled)
print(f'Baseline MLP Accuracy (raw features): {accuracy_score(y_test_clf, y_pred_mlp_clf_raw):.4f}')


# Baseline regressor: MLP on raw features (regression) with scaling
scaler_reg_raw = StandardScaler()
X_train_reg_scaled = scaler_reg_raw.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg_raw.transform(X_test_reg)
mlp_reg_raw = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
mlp_reg_raw.fit(X_train_reg_scaled, y_train_reg)
y_pred_mlp_reg_raw = mlp_reg_raw.predict(X_test_reg_scaled)
print(f'Baseline MLP R2 (raw features): {r2_score(y_test_reg, y_pred_mlp_reg_raw):.4f}')

Baseline MLP Accuracy (raw features): 0.9737
Baseline MLP R2 (raw features): 0.4787


/Users/Scott/Documents/Data Science/ADSWP/TabPFN/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [19]:
# Baseline classifier: SVM on raw features (classification) with scaling
from sklearn.svm import SVC
scaler_svm_clf_raw = StandardScaler()
X_train_clf_scaled_svm = scaler_svm_clf_raw.fit_transform(X_train_clf)
X_test_clf_scaled_svm = scaler_svm_clf_raw.transform(X_test_clf)
svm_clf_raw = SVC(kernel='rbf', random_state=42)
svm_clf_raw.fit(X_train_clf_scaled_svm, y_train_clf)
y_pred_svm_clf_raw = svm_clf_raw.predict(X_test_clf_scaled_svm)
print(f'Baseline SVM Accuracy (raw features): {accuracy_score(y_test_clf, y_pred_svm_clf_raw):.4f}')


# Enhanced classifier: SVM on TabPFN embeddings (classification) with scaling
scaler_svm_clf_emb = StandardScaler()
X_train_emb_scaled_svm_clf = scaler_svm_clf_emb.fit_transform(train_embeddings_clf)
X_test_emb_scaled_svm_clf = scaler_svm_clf_emb.transform(test_embeddings_clf)
svm_clf_emb = SVC(kernel='rbf', random_state=42)
svm_clf_emb.fit(X_train_emb_scaled_svm_clf, y_train_clf)
y_pred_svm_clf_emb = svm_clf_emb.predict(X_test_emb_scaled_svm_clf)
print(f'TabPFN Embedding + SVM Accuracy: {accuracy_score(y_test_clf, y_pred_svm_clf_emb):.4f}')


# Baseline regressor: SVR on raw features (regression) with scaling
from sklearn.svm import SVR
scaler_svr_reg_raw = StandardScaler()
X_train_reg_scaled_svr = scaler_svr_reg_raw.fit_transform(X_train_reg)
X_test_reg_scaled_svr = scaler_svr_reg_raw.transform(X_test_reg)
svr_reg_raw = SVR(kernel='rbf')
svr_reg_raw.fit(X_train_reg_scaled_svr, y_train_reg)
y_pred_svr_reg_raw = svr_reg_raw.predict(X_test_reg_scaled_svr)
print(f'Baseline SVR R2 (raw features): {r2_score(y_test_reg, y_pred_svr_reg_raw):.4f}')


# Enhanced regressor: SVR on TabPFN embeddings (regression) with scaling
scaler_svr_reg_emb = StandardScaler()
X_train_emb_scaled_svr_reg = scaler_svr_reg_emb.fit_transform(train_embeddings_reg)
X_test_emb_scaled_svr_reg = scaler_svr_reg_emb.transform(test_embeddings_reg)
svr_reg_emb = SVR(kernel='rbf')
svr_reg_emb.fit(X_train_emb_scaled_svr_reg, y_train_reg)
y_pred_svr_reg_emb = svr_reg_emb.predict(X_test_emb_scaled_svr_reg)
print(f'TabPFN Embedding + SVR R2: {r2_score(y_test_reg, y_pred_svr_reg_emb):.4f}')


# Baseline classifier: KNN on raw features (classification) with scaling
from sklearn.neighbors import KNeighborsClassifier
scaler_knn_clf_raw = StandardScaler()
X_train_clf_scaled_knn = scaler_knn_clf_raw.fit_transform(X_train_clf)
X_test_clf_scaled_knn = scaler_knn_clf_raw.transform(X_test_clf)
knn_clf_raw = KNeighborsClassifier(n_neighbors=5)
knn_clf_raw.fit(X_train_clf_scaled_knn, y_train_clf)
y_pred_knn_clf_raw = knn_clf_raw.predict(X_test_clf_scaled_knn)
print(f'Baseline KNN Accuracy (raw features): {accuracy_score(y_test_clf, y_pred_knn_clf_raw):.4f}')


# Enhanced classifier: KNN on TabPFN embeddings (classification) with scaling
scaler_knn_clf_emb = StandardScaler()
X_train_emb_scaled_knn_clf = scaler_knn_clf_emb.fit_transform(train_embeddings_clf)
X_test_emb_scaled_knn_clf = scaler_knn_clf_emb.transform(test_embeddings_clf)
knn_clf_emb = KNeighborsClassifier(n_neighbors=5)
knn_clf_emb.fit(X_train_emb_scaled_knn_clf, y_train_clf)
y_pred_knn_clf_emb = knn_clf_emb.predict(X_test_emb_scaled_knn_clf)
print(f'TabPFN Embedding + KNN Accuracy: {accuracy_score(y_test_clf, y_pred_knn_clf_emb):.4f}')


# Baseline regressor: KNN on raw features (regression) with scaling
from sklearn.neighbors import KNeighborsRegressor
scaler_knn_reg_raw = StandardScaler()
X_train_reg_scaled_knn = scaler_knn_reg_raw.fit_transform(X_train_reg)
X_test_reg_scaled_knn = scaler_knn_reg_raw.transform(X_test_reg)
knn_reg_raw = KNeighborsRegressor(n_neighbors=5)
knn_reg_raw.fit(X_train_reg_scaled_knn, y_train_reg)
y_pred_knn_reg_raw = knn_reg_raw.predict(X_test_reg_scaled_knn)
print(f'Baseline KNN R2 (raw features): {r2_score(y_test_reg, y_pred_knn_reg_raw):.4f}')


# Enhanced regressor: KNN on TabPFN embeddings (regression) with scaling
scaler_knn_reg_emb = StandardScaler()
X_train_emb_scaled_knn_reg = scaler_knn_reg_emb.fit_transform(train_embeddings_reg)
X_test_emb_scaled_knn_reg = scaler_knn_reg_emb.transform(test_embeddings_reg)
knn_reg_emb = KNeighborsRegressor(n_neighbors=5)
knn_reg_emb.fit(X_train_emb_scaled_knn_reg, y_train_reg)
y_pred_knn_reg_emb = knn_reg_emb.predict(X_test_emb_scaled_knn_reg)
print(f'TabPFN Embedding + KNN R2: {r2_score(y_test_reg, y_pred_knn_reg_emb):.4f}')

Baseline SVM Accuracy (raw features): 0.9825
TabPFN Embedding + SVM Accuracy: 0.3772
Baseline SVR R2 (raw features): 0.1822
TabPFN Embedding + SVR R2: -0.0000
Baseline KNN Accuracy (raw features): 0.9474
TabPFN Embedding + KNN Accuracy: 0.9737
Baseline KNN R2 (raw features): 0.4248
TabPFN Embedding + KNN R2: -0.2863
